# Agenda

1. Concurrency and parallelism in programming in general, and Python in particular
2. Basic threads
3. Joining threads
4. Switching threads and the GIL
5. Sharing data (and other resources)
6. Producer-consumer 
7. Events and timers
8. Multiprocessing
9. `concurrent.futures` and making life easier for ourselves

# Concurrency and parallelism in Python

- Concurency means: I have several things that I want to be tracking at once, even if they're not necessarily executing at once.
- Parallelism means: I have several things that I want to be tracking at once, *AND* they should also be executing at 